# Operationalize GovTech Benchmarking Framework

In [41]:
import requests
import pandas as pd
from getpass import getpass
from tqdm import tqdm 

### Set up the inference pipeline

In [42]:
# GovLLM-7B-ultra: Make sure to replace these values with your personal API URL and KEY
API_URL = "https://o0g65mtefl5fhjjw.us-east-1.aws.endpoints.huggingface.cloud"
API_KEY = getpass("Enter your API KEY:")

In [43]:
# # GEITje-7B-ultra: Make sure to replace these values with your personal API URL and KEY
# API_URL = "https://oi6h8u843v8nt5qt.eu-west-1.aws.endpoints.huggingface.cloud"
# API_KEY = getpass("Enter your API KEY:")

In [44]:
headers = {
	"Accept" : "application/json",
	"Authorization": "Bearer " + API_KEY,
    "Content-Type": "application/json" 
}

In [91]:
def query(prompt, parameters=None):
	if parameters is None:
		parameters = {
			"max_new_tokens": 128,
			"temperature": 0.5,
			"top_p": 0.99,
			"top_k": 50,
			"repetition_penalty": 1.,
			"wait_for_model": True,
			"do_sample": False
		}
			
	payload = {
		"inputs": prompt,
		"parameters": parameters
	}
	
	response = requests.post(API_URL, headers=headers, json=payload)
	# response successful? if not print the full error message
	response.raise_for_status()
	return response.json()

### Load framework

In [130]:
# prevent reading extra unnamed column
framework = pd.read_csv("../Results/framework_questions_translated.csv", usecols=[' #', '2022 GTMI Indicators & Sub-indicators NL', 'Response options & Data format NL'])


In [131]:
framework.head()

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL
0,I-1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ..."
1,I-1.1,Naam van het Overheids Cloud platform,Tekst
2,I-1.2,Cloud platform / strategie URL,URL
3,I-1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY
4,I-1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv..."


## Define functions to operationalize dataframe with answers and prompts

In [132]:
def get_main_indicator(index):
    if '.' in index:
        return index.split('.')[0]
    return index

In [133]:
def generate_prompt(role, indicator, sub_indicator, data_format):
	role = role + f"DATA FORMAT: {data_format} "
	if sub_indicator:
		question = f"{indicator}, indien ja, wat is de {sub_indicator} ?"
	else:
		question = f"{indicator} ?"
	prompt =  f"{role} \n\nVRAAG: {question} \n\nANTWOORD:"
	return prompt

In [134]:
def operationalize_framework(framework, role):
	framework_operationalized = framework.copy()
	framework_operationalized["Operationalisatie"] = None
	framework_operationalized['Prompt'] = None
	framework_operationalized[' #'] = framework_operationalized[' #'].str.replace('I-', '')

	for index, row in tqdm(framework_operationalized.iterrows(), total=framework_operationalized.shape[0]):
		idx = row[' #']
		main_indicator_idx = get_main_indicator(idx)
		sub_indicator_idx = idx.split('.')[1] if '.' in idx else None
		indicator_info = row['2022 GTMI Indicators & Sub-indicators NL']
		data_format = row['Response options & Data format NL']
				
		if sub_indicator_idx:
			# look up the main indicator
			main_indicator = framework_operationalized.loc[framework_operationalized[' #'] == main_indicator_idx]['2022 GTMI Indicators & Sub-indicators NL'].iloc[0]
			prompt = generate_prompt(role, main_indicator, indicator_info, data_format)
		else:
			prompt = generate_prompt(role, indicator_info, None, data_format)
		output = query(prompt)
		framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Operationalisatie'] = output[0]['generated_text']
		framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Prompt'] = prompt
		
	return framework_operationalized

### RUN THIS AWESOME FRAMEWORK OPERATIONALIZATION

In [135]:
role = "Je bent 'GovTech-GPT', een geavanceerde AI-assistent met uitgebreide expertise in digitale technologieën specifiek gericht op toepassingen binnen de Nederlandse overheid. Je belangrijkste taak is het ondersteunen bij het operationaliseren van e-gov benchmarking frameworks. Je antwoordt altijd op basis van de meest recente gegevens en inzichten, en houdt rekening met de specifieke context van de Nederlandse overheid. Antwoorden geef je alleen volgens het gespecificeerde dataformat, waarbij je, indien mogelijk, het cijfer gebruikt en niet de tekst. Voeg verder geen enkele tekst, toelichting of uitleg meer toe. Als je het antwoord niet weet, geef je geen fictieve informatie of uitleg, maar antwoord enkel en alleen met: 'Geen antwoord.'  \n\n"

In [136]:
# # test with small sample
# framework = framework.head()

In [137]:
df = operationalize_framework(framework, role)

100%|██████████| 351/351 [11:15<00:00,  1.92s/it]


In [138]:
df.head()

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,Operationalisatie,Prompt
0,1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ...","Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
1,1.1,Naam van het Overheids Cloud platform,Tekst,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
2,1.2,Cloud platform / strategie URL,URL,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
3,1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
4,1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv...","Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."


In [128]:
df2

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,Operationalisatie,Prompt
0,1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ...","Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
1,1.1,Naam van het Overheids Cloud platform,Tekst,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
2,1.2,Cloud platform / strategie URL,URL,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
3,1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY,"Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."
4,1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv...","Je bent 'GovTech-GPT', een geavanceerde AI-ass...","Je bent 'GovTech-GPT', een geavanceerde AI-ass..."


In [129]:
for index, row in df2.iterrows():
	print(f"PROMPT: {row['Prompt']}")
	print(f"{row['Operationalisatie']}")
	print("\n")

PROMPT: Je bent 'GovTech-GPT', een geavanceerde AI-assistent met uitgebreide expertise in digitale technologieën specifiek gericht op toepassingen binnen de Nederlandse overheid. Je belangrijkste taak is het ondersteunen bij het operationaliseren van e-gov benchmarking frameworks. Je antwoordt altijd op basis van de meest recente gegevens en inzichten, en houdt rekening met de specifieke context van de Nederlandse overheid. Antwoorden geef je alleen volgens het gespecificeerde dataformat, waarbij je, indien mogelijk, het cijfer gebruikt en niet de tekst. Voeg verder geen enkele tekst, toelichting of uitleg meer toe. Als je het antwoord niet weet, geef je geen fictieve informatie of uitleg, maar antwoord enkel en alleen met: 'Geen antwoord.'  

DATA FORMAT: 0= Nee, 1= Alleen cloud strategie/beleid (nog geen platform), 2= Ja (platform in gebruik)  

VRAAG: Is er een gedeeld cloud platform beschikbaar voor alle overheids organisaties? ? 

ANTWOORD:
Je bent 'GovTech-GPT', een geavanceerde 

# Save data to CSV file

In [139]:
df.to_csv("../Results/operationalization-PoF.csv", index=False)

In [108]:
df.head()

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,Operationalisatie,Prompt
0,1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ...",Antwoord volgens het data-format. DATA FORMAT:...,Antwoord volgens het data-format. DATA FORMAT:...
1,1.1,Naam van het Overheids Cloud platform,Tekst,Antwoord volgens het data-format. DATA FORMAT:...,Antwoord volgens het data-format. DATA FORMAT:...
2,1.2,Cloud platform / strategie URL,URL,Antwoord volgens het data-format. DATA FORMAT:...,Antwoord volgens het data-format. DATA FORMAT:...
3,1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY,Antwoord volgens het data-format. DATA FORMAT:...,Antwoord volgens het data-format. DATA FORMAT:...
4,1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv...",Antwoord volgens het data-format. DATA FORMAT:...,Antwoord volgens het data-format. DATA FORMAT:...
